# DeepState - Experiments

- Using DeepState with gluonts to run some experiments
- Follow this installation guide: http://gluon-ts-staging.s3-accelerate.dualstack.amazonaws.com/PR-271/4/install.html

Important note! Some of these models are not yet released in version "0.3.3" which is currently the most recent version on pypi from where you install packages via `pip install`. Hence, you should follow **Install from Source Code** (which I did) **Install from Github**

### Deep State Space Models (Rangapuram et al., 2018)

**DeepState** combines state space models (SSM) with a recurrent neural network (RNN). The SSM is applied *locally* to the individual time series that is parametrized using a *global* representation in form of a RNN. The RNN is trained on the entire dataset. 

- [`DeepStateEstimator`](http://gluon-ts.mxnet.io/master/api/gluonts/gluonts.model.deepstate.html#gluonts.model.deepstate.DeepStateEstimator)


In [1]:
# if version does not include DeepState, uncomment the following:
!pip install git+https://github.com/awslabs/gluon-ts.git
# !pip install mxnet-cu101

import gluonts
print(gluonts.__version__)

# standard imports
import numpy as np
import pandas as pd

# vis imports 
%matplotlib inline
import matplotlib.pyplot as plt

# json
import json

# gluon data 
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# model imports 
from gluonts.model.deepstate import DeepStateEstimator
# from gluonts.model.deep_factor import DeepFactorEstimator
# from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint

  Cloning https://github.com/awslabs/gluon-ts.git to /tmp/pip-req-build-ddvttyv_
  Installing build dependencies ... done
    100% |████████████████████████████████| 81kB 2.0MB/s ta 0:00:011
    100% |████████████████████████████████| 28.4MB 1.8MB/s eta 0:00:011  27% |████████▉                       | 7.8MB 58.0MB/s eta 0:00:01    95% |██████████████████████████████▍ | 27.0MB 84.4MB/s eta 0:00:01
    100% |████████████████████████████████| 10.4MB 8.4MB/s  eta 0:00:01   46% |██████████████▉                 | 4.8MB 86.5MB/s eta 0:00:01
    100% |████████████████████████████████| 4.8MB 14.2MB/s ta 0:00:011
    100% |████████████████████████████████| 61kB 43.3MB/s ta 0:00:01
    100% |████████████████████████████████| 194kB 50.2MB/s ta 0:00:01
  Running setup.py bdist_wheel for gluonts ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-5sqzk37p/wheels/09/ba/b9/c830628df4d977a9dfe661f6001cdb861fc3f2524bac0ee433
  Running setup.py bdist_wheel for holidays ... done
  Stored in directo

INFO:root:Using CPU


In [2]:
def deep_state(seed=42, data="m4_daily", epochs=25, batches=50):
    dataset = get_dataset(data)

    mx.random.seed(seed)
    np.random.seed(seed)
    
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=epochs,
        num_batches_per_epoch=batches,
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepStateEstimator(
        trainer=trainer,
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
#         past_length=48,
        use_feat_static_cat=True,
        cardinality=[cardinality],
    )

    predictor = estimator.train(dataset.train)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}

    pprint(output)

# if __name__ == "__main__":
#     deep_state()

In [ ]:
%%time
if __name__ == "__main__":
    deep_state(seed=42, data="m4_daily", epochs=25, batches=50)

INFO:root:downloading and processing m4_daily


saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily/train/data.json
saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily/test/data.json


INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [06:08<00:00,  7.38s/it, avg_epoch_loss=-1.4]    
INFO:root:Epoch[0] Elapsed time 368.850 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=-1.396312
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [01:57<00:00,  2.35s/it, avg_epoch_loss=0.0665]
INFO:root:Epoch[1] Elapsed time 117.448 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=0.066499
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [01:56<00:00,  2.32s/it, avg_epoch_loss=-1.24]
INFO:root:Epoch[2] Elapsed time 116.127 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=-1.238061
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [01:57<00:00,  2.34s/it, avg_epoch_loss=-1.52]
INFO:root:Epoch[3] Elapsed time 117.205 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=-1.518090
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 50/50 [01:58<00:00,  2.37s/

In [3]:
%%time
if __name__ == "__main__":
    deep_state(seed=42, data="m4_daily", epochs=35, batches=50)

INFO:root:downloading and processing m4_daily


saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily/train/data.json
saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily/test/data.json


INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [05:57<00:00,  7.14s/it, avg_epoch_loss=-1.4]    
INFO:root:Epoch[0] Elapsed time 357.026 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=-1.396312
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [01:58<00:00,  2.36s/it, avg_epoch_loss=0.0665]
INFO:root:Epoch[1] Elapsed time 118.125 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=0.066499
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [01:57<00:00,  2.34s/it, avg_epoch_loss=-1.24]
INFO:root:Epoch[2] Elapsed time 117.085 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=-1.238061
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [01:57<00:00,  2.36s/it, avg_epoch_loss=-1.52]
INFO:root:Epoch[3] Elapsed time 117.825 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=-1.518090
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 50/50 [01:58<00:00,  2.37s/

{'MASE': 3.95988776,
 'MSIS': 71.02142783,
 'sMAPE': 0.03622678,
 'wQuantileLoss[0.5]': 0.033905,
 'wQuantileLoss[0.9]': 0.02346546}
CPU times: user 3h 7min 52s, sys: 3min 13s, total: 3h 11min 5s
Wall time: 1h 25min 16s


In [4]:
%%time
if __name__ == "__main__":
    deep_state(seed=42, data="m4_daily", epochs=45, batches=50)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [06:09<00:00,  7.38s/it, avg_epoch_loss=-1.4]    
INFO:root:Epoch[0] Elapsed time 369.019 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=-1.396312
INFO:root:Epoch[1] Learning rate is 0.001
 10%|█         | 5/50 [00:14<02:12,  2.95s/it, avg_epoch_loss=-1.23]


KeyboardInterrupt: 

In [ ]:
%%time
if __name__ == "__main__":
    deep_state(seed=43, data="m4_daily", epochs=35, batches=50)

In [ ]:
%%time
if __name__ == "__main__":
    deep_state(seed=43, data="m4_daily", epochs=45, batches=50)

In [ ]:
%%time
if __name__ == "__main__":
    deep_state(seed=44, data="m4_daily", epochs=35, batches=50)

In [ ]:
%%time
if __name__ == "__main__":
    deep_state(seed=44, data="m4_daily", epochs=45, batches=50)